**実習ファイルをアップロードして開いたら、一番上のファイル名「クラス番号氏名」を必ず自分のものに書き換えること。**

# 実習04-2 CSVファイル読み込みとアルゴリズム選択（分類）
---
次に、分類問題について、次の手順でアルゴリズムの選択を行う。

0.  実行したいタスクと学習分類の確認
1.   学習用データをCSVファイルから読み込み、データを準備する。
2.   全ての選択可能なアルゴリズムについて学習を行い、精度の高そうなモデルを選択する。
3.   選択したアルゴリズムを使用して再びモデルを作成し、訓練用データで訓練した後、テスト用データで精度を確認する。


## 0. 実行したいタスクと学習分類の確認

ここでは、ワインの情報（アルコール度数や色の濃さなど13項目）から、ワインの種類（class_0、class_1、class_2）がどれかを予測するAIを作成する。使用するデータセットに予測すべき値が含まれており、予測は3つのうちどれかに分類するため、学習分類は

* 教師あり学習
* 分類（多値分類）

である。

## 1. データセットの準備
### 1-1. CSVファイルのアップロードと読み込み
分類問題には、配布するワインのデータセット（wine.csv）を使用し、
CSVファイルの読み込みには、pandasモジュールのread_csv()を使用する。

```
←まず、CVSファイルを開いて中身を確認し、Google Colabにファイルをアップロードする。
```

```
# アップロード後
import pandas as pd
df_wine = pd.read_csv('ファイル名')
```


In [5]:
# CSVファイルの読み込み
import pandas as pd
df_wine = pd.read_csv('wine.csv')
# 表示して確認してもよい
df_wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,class_0
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,class_0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,class_0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,class_0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,class_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,class_2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,class_2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,class_2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,class_2


#### 1-2. 入力データと出力データの作成
入力データとして、最初の13列（最後の「class」以外）を使用する。
```
# 入力データに使用する列を抽出（最初の13列にする、最後の1列を削除）
変数名 = df_wine.iloc[:,a:b] # 行は全て、列はa列目からb-1列目までにする
または
変数名 = df_wine.drop('削除する列名', axis=1) # 列の場合はaxis=1
など

```

In [6]:
# 入力データに使用する列を抽出する(最初の13列を取るか、最後の1列を削除)
x = df_wine.iloc[:,:13]
# 表示して確認してもよい
x

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


出力に必要な列のみ抽出して出力データyを作
成する。
```
#抽出（1列なので列名の指定が簡単）
y = df_wine['列名']

```

In [7]:
# 抽出
y = df_wine['class']
# 表示して確認してもよい
y

,class
0,class_0
1,class_0
2,class_0
3,class_0
4,class_0
...,...
173,class_2
174,class_2
175,class_2
176,class_2


### 1-3. 訓練用とテスト用に分ける
入力データXと出力データyができたら、訓練用データとテストデータに分割する。train_test_split()を使用する。

**random_stateには学籍番号を入れてください**

※分類問題では、出力データのクラス割合に応じて分割（層化）をする場合がある。（stratify=y）

```
# インポート
from sklearn.model_selection import train_test_split
# 訓練用データセットとテスト用データセットへの分割（訓練用80%、テスト用20%）、stratify=y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=学籍番号, stratify=y)
```

In [8]:
# データセットを分割する関数の読み込み
from sklearn.model_selection import train_test_split
# 訓練用データセットとテスト用データセットへの分割（訓練用70%、テスト用30%）
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2220042, stratify=y)

## 2. アルゴリズムの選択
### 2-1. 使用可能な全てのアルゴリズムの取得
sklearnで使用可能なアルゴリズム一覧を取得するには、sklearn.utils.discovery.all_estimatorsが使用できる。 今回は教師あり学習の分類問題に該当するため、type_filter="classifier"を指定する。

```
# 使用可能なアルゴリズムの取得
from sklearn.utils.discovery import all_estimators
all_clf = all_estimators(type_filter="classifier")
# 表示して確認してもよい

```

In [9]:
#使用可能なアルゴリズムの取得
from sklearn.utils.discovery import all_estimators
# 表示して確認してもよい
all_est = all_estimators(type_filter="classifier")

### 2-2. 各アルゴリズムの性能比較
それぞれのアルゴリズムに対して、

*   モデル作成
*   scoreを計算

を実施し、forループで繰り返す。


```
import warnings

#警告を非表示にする
warnings.simplefilter('ignore')

for(name, algo) in all_clf:
  try:
    model = algo()
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(name,"のscore＝",score)

  # エラーが出ても止まらないようにする
  except Exception as e:
    pass

# 警告表示を元に戻す
warnings.simplefilter('default')
```




In [10]:
# forループを使用して各アルゴリズムのscoreを表示する
# scoreが最大になるアルゴリズムをプログラム上で探してもよい
import warnings

# 警告を非表示にする
warnings.simplefilter('ignore')

for(name, algo) in all_est:
  try:
    model = algo()
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    if score > 0.7:
      print(name,"のscore＝",score)

  # エラーが出ても止まらないようにする
  except Exception as e:
    pass

# 警告表示を元に戻す
warnings.simplefilter('default')

AdaBoostClassifier のscore＝ 0.9629629629629629
BaggingClassifier のscore＝ 0.9814814814814815
CalibratedClassifierCV のscore＝ 0.9629629629629629
ComplementNB のscore＝ 0.7037037037037037
DecisionTreeClassifier のscore＝ 0.9074074074074074
ExtraTreeClassifier のscore＝ 0.9444444444444444
ExtraTreesClassifier のscore＝ 1.0
GaussianNB のscore＝ 0.9814814814814815
GradientBoostingClassifier のscore＝ 0.9814814814814815
HistGradientBoostingClassifier のscore＝ 0.9814814814814815
LinearDiscriminantAnalysis のscore＝ 1.0
LinearSVC のscore＝ 0.9444444444444444
LogisticRegression のscore＝ 0.9444444444444444
LogisticRegressionCV のscore＝ 0.9814814814814815
MultinomialNB のscore＝ 0.8518518518518519
NearestCentroid のscore＝ 0.7777777777777778
NuSVC のscore＝ 0.9074074074074074
QuadraticDiscriminantAnalysis のscore＝ 1.0
RandomForestClassifier のscore＝ 1.0
RidgeClassifier のscore＝ 1.0
RidgeClassifierCV のscore＝ 1.0
SVC のscore＝ 0.7037037037037037


#### 2-3. モデルを選択する
上記の結果から、精度が良さそうなモデルを選択する。

**[課題] 選択したモデルと簡単な説明を記入してください。**
*   **選択したモデル（分類）：[ RidgeClassifier ]**
*   **モデルの簡単な説明：[ 主に多クラス分類問題に対応しており、L2正則化を用いることで過学習を防ぎ、モデルの汎化能力を向上させます。 ]**



## 3. モデルの作成
選択したモデルについて、すべての学習データで学習し、テストデータでscoreを出してみよう。
モデルを作成する際は、直接クラスを指定するか、[scikit-learnの公式](https://scikit-learn.org/stable/)などで、import元の場所を調べる。

**モデル作成時に、random_stateを学籍番号に設定すること。**<br>
**警告が出る場合も、とりあえずは気にしなくていいです。**

In [15]:
# モデルのインポート、作成、fit、score
from sklearn.linear_model import RidgeClassifier
model = RidgeClassifier(random_state=2220042)
model.fit(x, y)
score = model.score(X_test, y_test)
score

1.0

## 提出物など

*   テキストブロックへの追記（選択したモデル）
*   コードブロックの記入と実行（特に、最後のscoreの値）

が確認出来たら、**教員に結果（選択したモデルと最後のscore）を確認してもらうこと。** その後、ファイルが保存されているかを確認し、「ファイル＞ダウンロード＞.ipynbをダウンロード」を順にクリックして.ipynbファイルをダウンロードする。

**ダウンロードしたipynbファイルを指定の場所に提出してください。**
